In [1]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [2]:
def preprocessed_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r"[^a-zA-Z]", " ", text)
    
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Join tokens back into a single string
    preprocessed_text = " ".join(tokens)
    
    return preprocessed_text

In [3]:
# Read data 
data = pd.read_csv("Survey Data_shipping.csv", encoding='latin-1')

In [4]:
data.columns

Index(['ï»¿OCF Journey',
       'Response Ticker - Why you gave this number in the survey',
       'Why - issue Category', 'Why - Keywords/Words related to issue',
       'Why - # of Words', 'Response Ticker - What we could do better?',
       'What - Issue', 'What - Words related to issue', 'What - # of Words',
       'Application area', 'Comment_Metadata1', 'Comment_Metadata2',
       'Q1 - How likely are you to recommend UPS to a friend or colleague?',
       'Survey Date (+00:00 GMT)', 'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Preprocess the text column
data["preprocessed_text"] = data["Response Ticker - Why you gave this number in the survey"].apply(preprocessed_text)

In [7]:
data['Why - issue Category'].value_counts()

Operational                   2147
Layout/Navigation/Usablity    1973
Others                         595
Pricing/Cost                   525
Support/Sales                  430
Technical/Performance          224
others                          52
Operational                      5
Name: Why - issue Category, dtype: int64

In [8]:
data['Why - issue Category'] = data['Why - issue Category'].replace({'Operational ':'Operational','others': 'Others'})

In [9]:
data['Why - issue Category'].value_counts()

Operational                   2152
Layout/Navigation/Usablity    1973
Others                         647
Pricing/Cost                   525
Support/Sales                  430
Technical/Performance          224
Name: Why - issue Category, dtype: int64

In [49]:
# Split the data into features (X) and labels (y)
X = data["preprocessed_text"]
y = data["Why - issue Category"]  

In [44]:
import numpy as np

In [48]:
len(np.unique(y))

6

In [50]:

# Perform label encoding on the class labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Vectorize the text using Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max([len(x) for x in X_train])
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

# Define the CNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16)


# Predict classes using the trained model
y_pred_probs = model.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# Calculate the accuracy of predicted classes
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate the prediction accuracy for each class
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)



Epoch 1/10
298/298 [==============================] - 7s 20ms/step - loss: 1.2152 - accuracy: 0.5557 - val_loss: 0.8478 - val_accuracy: 0.6751
Epoch 2/10
298/298 [==============================] - 5s 16ms/step - loss: 0.7476 - accuracy: 0.7433 - val_loss: 0.6766 - val_accuracy: 0.7464
Epoch 3/10
298/298 [==============================] - 5s 16ms/step - loss: 0.5344 - accuracy: 0.8107 - val_loss: 0.6743 - val_accuracy: 0.7523
Epoch 4/10
298/298 [==============================] - 5s 15ms/step - loss: 0.4177 - accuracy: 0.8584 - val_loss: 0.6944 - val_accuracy: 0.7490
Epoch 5/10
298/298 [==============================] - 5s 15ms/step - loss: 0.3231 - accuracy: 0.8880 - val_loss: 0.7842 - val_accuracy: 0.7557
Epoch 6/10
298/298 [==============================] - 5s 15ms/step - loss: 0.2659 - accuracy: 0.9099 - val_loss: 0.7864 - val_accuracy: 0.7498
Epoch 7/10
298/298 [==============================] - 5s 15ms/step - loss: 0.2223 - accuracy: 0.9261 - val_loss: 0.9190 - val_accuracy: 0.7431

# Testing model accuracy

In [105]:
test_data = pd.read_csv('CXShieldDigitalSurveys.csv')

In [106]:
test_data.head()

,Start Date (+00:00 GMT),Survey Date (+00:00 GMT),Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date (+00:00 GMT),Recipient Last Name,Recipient First Name,...,External Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,NPS Group,Q1 - How likely are you to recommend UPS to a friend or colleague?,Q2 - To what extent did your most recent [journey experience] experience influen...,Q3 - Why?,Why - Issue
0,4/19/2023 13:17,4/19/2023 13:18,IP Address,208.127.65.83,100,60,True,4/19/2023 13:18,NaN,NaN,...,NaN,38.791100,-77.526400,email,EN,Detractor,6,Increased a little,The criteria needed.,NaN
1,4/19/2023 12:39,4/19/2023 12:40,IP Address,50.195.184.141,100,45,True,4/19/2023 12:40,NaN,NaN,...,NaN,40.211600,-85.427300,email,EN,Detractor,0,Decreased a lot,"UPS never followed up, never contacted us in a...",NaN
2,4/19/2023 12:33,4/19/2023 12:34,IP Address,166.70.168.63,100,60,True,4/19/2023 12:34,NaN,NaN,...,NaN,40.007200,-111.603100,email,EN,Detractor,0,Decreased a lot,Horrible experience. Still not resolved at all...,NaN
3,4/19/2023 12:03,4/19/2023 12:07,IP Address,173.187.41.221,100,234,True,4/19/2023 12:07,NaN,NaN,...,NaN,31.182300,-83.764100,email,EN,Detractor,5,Decreased a lot,Network Server crashed and UPS will not totall...,NaN
4,2/23/2021 14:20,2/23/2021 14:22,IP Address,71.206.130.189,100,103,True,2/23/2021 14:22,NaN,NaN,...,NaN,37.463806,-77.398003,email,EN,Promoter,10,Increased a lot,"Easy process, and the UPS associate was very h...",NaN


In [107]:
test_series = test_data[['Q3 - Why?']]

In [108]:
test_series= test_series.dropna()

In [109]:
test_series['preprocessed_text']= test_series['Q3 - Why?'].apply(preprocessed_text )

In [97]:
test_series.head()

,Q3 - Why?,preprocessed_text
0,The criteria needed.,criteria need
1,"UPS never followed up, never contacted us in a...",up never follow never contact us way never anyth
2,Horrible experience. Still not resolved at all...,horribl experi still resolv up lost item still...
3,Network Server crashed and UPS will not totall...,network server crash up total uninstal let us ...
4,"Easy process, and the UPS associate was very h...",easi process up associ help


In [98]:
test_s = test_series['preprocessed_text']

In [101]:
X_test.shape, X_train.shape, test_s.shape

((1191, 200), (4760, 200), (27905,))

In [112]:
#preprocesing
test_s = tokenizer.texts_to_sequences(test_s)

# Pad sequences to have the same length
max_length = max([len(x) for x in X_train])
test_s = pad_sequences(test_s, maxlen=max_length)

In [113]:
test_s.shape, X_train.shape

((27905, 200), (4760, 200))

In [115]:
y_pred_test = model.predict(test_s)

873/873 [==============================] - 3s 4ms/step


In [118]:
y_pred_deploy = y_pred_test.argmax(axis=1)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [122]:
test_series['predicted_class'] = y_pred_deploy

In [130]:
y_pred_deploy_labeled = label_encoder.inverse_transform(y_pred_deploy)

In [131]:
test_series['y_pred_labelled'] = y_pred_deploy_labeled

In [132]:
test_series

,Q3 - Why?,preprocessed_text,predicted_class,y_pred_labelled
0,The criteria needed.,criteria need,0,Layout/Navigation/Usablity
1,"UPS never followed up, never contacted us in a...",up never follow never contact us way never anyth,0,Layout/Navigation/Usablity
2,Horrible experience. Still not resolved at all...,horribl experi still resolv up lost item still...,0,Layout/Navigation/Usablity
3,Network Server crashed and UPS will not totall...,network server crash up total uninstal let us ...,0,Layout/Navigation/Usablity
4,"Easy process, and the UPS associate was very h...",easi process up associ help,0,Layout/Navigation/Usablity
...,...,...,...,...
34585,reliable service with the nicest UPS driver in...,reliabl servic nicest up driver world,0,Layout/Navigation/Usablity
34586,"When I have my regular UPS man Tom, everything...",regular up man tom everyth fine fantast job va...,0,Layout/Navigation/Usablity
34587,Uploading/merging a mail list into WorldShip i...,upload merg mail list worldship easi watch vid...,0,Layout/Navigation/Usablity
34588,Worldship is far too complex to use. I have b...,worldship far complex use use websit year cont...,0,Layout/Navigation/Usablity
